# Processing a WLD output with the DM stack

In this notebook we use part of the DM stack to detect, deblend and measure images produced with the WeakLensingDeblending package. In order to use this notebook you need to have the LSST DM-stack available (you can use the common stack installed at NERSC used in the kernel file in this repository)

In [1]:
import lsst.afw.table
import lsst.afw.image
import lsst.afw.math
import lsst.meas.algorithms
import lsst.meas.base
import lsst.meas.deblender
import numpy as np
import astropy.io.fits as fits
import descwl
import astropy.table
import scipy.spatial
import scipy.ndimage
from lsst.sims.GalSimInterface.wcsUtils import tanSipWcsFromDetector
from lsst.sims.GalSimInterface import LSSTCameraWrapper
from lsst.sims.utils import ObservationMetaData
schema = lsst.afw.table.SourceTable.makeMinimalSchema()
config1 = lsst.meas.algorithms.SourceDetectionConfig()
min_pix = 1
bkg_bin_size = 32
thr_value = 5
hsm = False
# Tweaks in the configuration that can improve detection
# Change carefully!
#####
config1.tempLocalBackground.binSize=bkg_bin_size # This changes the local background binning. The default is 32 pixels
config1.minPixels=min_pix # This changes the minimum size of a source. The default is 1
config1.thresholdValue=thr_value # This changes the detection threshold for the footprint (5 is the default)
#####
detect = lsst.meas.algorithms.SourceDetectionTask(schema=schema, config=config1)
deblend = lsst.meas.deblender.SourceDeblendTask(schema=schema)
config1 = lsst.meas.base.SingleFrameMeasurementConfig()
## HSM is not included in the stack by default. You have to download it and activate it.
if hsm:
    import lsst.meas.extensions.shapeHSM
    config1.plugins.names.add('ext_shapeHSM_HsmShapeBj')
    config1.plugins.names.add('ext_shapeHSM_HsmShapeLinear')
    config1.plugins.names.add('ext_shapeHSM_HsmShapeKsb')
    config1.plugins.names.add('ext_shapeHSM_HsmShapeRegauss')
    config1.plugins.names.add('ext_shapeHSM_HsmSourceMoments')
    config1.plugins.names.add('ext_shapeHSM_HsmPsfMoments')
measure = lsst.meas.base.SingleFrameMeasurementTask(schema=schema, config=config1)
camera_wrapper = LSSTCameraWrapper()
obs = ObservationMetaData(pointingRA=0, pointingDec=0,
                        boundType='circle', boundLength=2.0,
                        mjd=52000.0, rotSkyPos=0,
                        bandpassName='i')

In [2]:
def process(input_path, output_path=None,seed=123):
    LSST_i = descwl.output.Reader(input_path).results # We read the image using descwl's package
    LSST_i.add_noise(noise_seed=seed) # We add noise
    image = lsst.afw.image.ImageF(LSST_i.survey.image.array) # We translate the image to be stack-readable
    sky_magnitude = LSST_i.survey.sky_brightness + LSST_i.survey.extinction*(LSST_i.survey.airmass -1.2)
    sky_counts = LSST_i.survey.exposure_time*LSST_i.survey.zero_point*10**(-0.4*(sky_magnitude-24))*LSST_i.survey.pixel_scale**2
    variance_array = LSST_i.survey.image.array+sky_counts # We generate a variance array
    variance = lsst.afw.image.ImageF(variance_array) # Generate the variance image
    masked_image = lsst.afw.image.MaskedImageF(image, None, variance) # Generate a masked image, i.e., an image+mask+variance image (with mask=None)
    psf_array = LSST_i.survey.psf_image.array # We read the PSF image from the package
    psf_array = psf_array.astype(np.float64) 
    psf_new = scipy.ndimage.zoom(psf_array,zoom=75/76.) # We have to rescale to have odd dimensions
    im = lsst.afw.image.ImageD(psf_new) # Convert to stack's format
    fkernel = lsst.afw.math.FixedKernel(im) 
    psf = lsst.meas.algorithms.KernelPsf(fkernel) # Create the kernel in the stack's format
    exposure = lsst.afw.image.ExposureF(masked_image) # Passing the image to the stack
    exposure.setPsf(psf) # Assign the exposure the PSF that we created
    wcs_in = tanSipWcsFromDetector('R:2,2 S:1,1',camera_wrapper,obs,2000) # We generate a WCS
    exposure.setWcs(wcs_in) # And assign it to the exposure
    table = lsst.afw.table.SourceTable.make(schema)  # this is really just a factory for records, not a table
    detect_result = detect.run(table, exposure) # We run the stack (the detection task)
    catalog = detect_result.sources   # this is the actual catalog, but most of it's still empty
    deblend.run(exposure, catalog) # run the deblending task
    measure.run(catalog, exposure) # run the measuring task
    catalog = catalog.copy(deep=True)
    if output_path is not None:
        catalog.writeFits(output_path) #write a copy of the catalog
    return catalog # We return a catalog object

Define the path where the input catalogs are and where you want to write the outputs

In [3]:
import os
btf_dir = '/global/projecta/projectdirs/lsst/groups/WL/projects/wl-btf/'

In [4]:
catalog = process(os.path.join(btf_dir,'LSST_i_lite.fits'), output_path='LSST_i_DM.fits.gz')

Let's check the catalog:

In [5]:
catalog.schema

Schema(
    (Field['L'](name="id", doc="unique ID"), Key<L>(offset=0, nElements=1)),
    (Field['Angle'](name="coord_ra", doc="position in ra/dec"), Key<Angle>(offset=8, nElements=1)),
    (Field['Angle'](name="coord_dec", doc="position in ra/dec"), Key<Angle>(offset=16, nElements=1)),
    (Field['L'](name="parent", doc="unique ID of parent source"), Key<L>(offset=24, nElements=1)),
    (Field['I'](name="deblend_nChild", doc="Number of children this object has (defaults to 0)"), Key<I>(offset=32, nElements=1)),
    (Field['Flag'](name="deblend_deblendedAsPsf", doc="Deblender thought this source looked like a PSF"), Key['Flag'](offset=40, bit=0)),
    (Field['D'](name="deblend_psfCenter_x", doc="If deblended-as-psf, the PSF centroid", units="pixel"), Key<D>(offset=48, nElements=1)),
    (Field['D'](name="deblend_psfCenter_y", doc="If deblended-as-psf, the PSF centroid", units="pixel"), Key<D>(offset=56, nElements=1)),
    (Field['D'](name="deblend_psfFlux", doc="If deblended-as-psf, the

As we see the catalog contains information about position, flux (using the SDSS algorithms), shape , their respective uncertainties and numerous flags. It also includes `blendedness`, and `extendedness` parameters.

In [9]:
!ls -lrh /global/projecta/projectdirs/lsst/groups/WL/projects/wl-btf/*.fits*

-rw-rw----+ 1 jsanch87 lsst     125M Jan 22 13:36 /global/projecta/projectdirs/lsst/groups/WL/projects/wl-btf/OneDegSq.fits
-rw-rw----+ 1 jsanch87 lsst      72M Jan 24 14:13 /global/projecta/projectdirs/lsst/groups/WL/projects/wl-btf/LSST_i_lite.fits
-rw-r--r--  1 jsanch87 jsanch87  41M Feb  7 22:23 /global/projecta/projectdirs/lsst/groups/WL/projects/wl-btf/LSST_i_DM.fits.gz
-rw-rw----+ 1 jsanch87 lsst     851M Jan 22 16:57 /global/projecta/projectdirs/lsst/groups/WL/projects/wl-btf/LSST_i.fits


In [7]:
tab = catalog.asAstropy() #We can also convert the catalog to an astropy table and show the contents
tab

id,coord_ra,coord_dec,parent,deblend_nChild,deblend_deblendedAsPsf,deblend_psfCenter_x,deblend_psfCenter_y,deblend_psfFlux,deblend_tooManyPeaks,deblend_parentTooBig,deblend_masked,deblend_skipped,deblend_rampedTemplate,deblend_patchedTemplate,deblend_hasStrayFlux,base_GaussianCentroid_x,base_GaussianCentroid_y,base_GaussianCentroid_flag,base_GaussianCentroid_flag_noPeak,base_GaussianCentroid_flag_resetToPeak,base_NaiveCentroid_x,base_NaiveCentroid_y,base_NaiveCentroid_flag,base_NaiveCentroid_flag_noCounts,base_NaiveCentroid_flag_edge,base_NaiveCentroid_flag_resetToPeak,base_SdssCentroid_x,slot_Centroid_x,base_SdssCentroid_y,slot_Centroid_y,base_SdssCentroid_xSigma,slot_Centroid_xSigma,base_SdssCentroid_ySigma,slot_Centroid_ySigma,base_SdssCentroid_flag,base_CircularApertureFlux_flag_badCentroid,base_GaussianCentroid_flag_badInitialCentroid,base_GaussianFlux_flag_badCentroid,base_NaiveCentroid_flag_badInitialCentroid,base_PsfFlux_flag_badCentroid,base_SdssShape_flag_badCentroid,base_Variance_flag_badCentroid,slot_Centroid_flag,base_SdssCentroid_flag_edge,base_CircularApertureFlux_flag_badCentroid_edge,base_GaussianCentroid_flag_badInitialCentroid_edge,base_GaussianFlux_flag_badCentroid_edge,base_NaiveCentroid_flag_badInitialCentroid_edge,base_PsfFlux_flag_badCentroid_edge,base_SdssShape_flag_badCentroid_edge,base_Variance_flag_badCentroid_edge,slot_Centroid_flag_edge,base_SdssCentroid_flag_noSecondDerivative,base_CircularApertureFlux_flag_badCentroid_noSecondDerivative,base_GaussianCentroid_flag_badInitialCentroid_noSecondDerivative,base_GaussianFlux_flag_badCentroid_noSecondDerivative,base_NaiveCentroid_flag_badInitialCentroid_noSecondDerivative,base_PsfFlux_flag_badCentroid_noSecondDerivative,base_SdssShape_flag_badCentroid_noSecondDerivative,base_Variance_flag_badCentroid_noSecondDerivative,slot_Centroid_flag_noSecondDerivative,base_SdssCentroid_flag_almostNoSecondDerivative,base_CircularApertureFlux_flag_badCentroid_almostNoSecondDerivative,base_GaussianCentroid_flag_badInitialCentroid_almostNoSecondDerivative,base_GaussianFlux_flag_badCentroid_almostNoSecondDerivative,base_NaiveCentroid_flag_badInitialCentroid_almostNoSecondDerivative,base_PsfFlux_flag_badCentroid_almostNoSecondDerivative,base_SdssShape_flag_badCentroid_almostNoSecondDerivative,base_Variance_flag_badCentroid_almostNoSecondDerivative,slot_Centroid_flag_almostNoSecondDerivative,base_SdssCentroid_flag_notAtMaximum,base_CircularApertureFlux_flag_badCentroid_notAtMaximum,base_GaussianCentroid_flag_badInitialCentroid_notAtMaximum,base_GaussianFlux_flag_badCentroid_notAtMaximum,base_NaiveCentroid_flag_badInitialCentroid_notAtMaximum,base_PsfFlux_flag_badCentroid_notAtMaximum,base_SdssShape_flag_badCentroid_notAtMaximum,base_Variance_flag_badCentroid_notAtMaximum,slot_Centroid_flag_notAtMaximum,base_SdssCentroid_flag_resetToPeak,base_CircularApertureFlux_flag_badCentroid_resetToPeak,base_GaussianCentroid_flag_badInitialCentroid_resetToPeak,base_GaussianFlux_flag_badCentroid_resetToPeak,base_NaiveCentroid_flag_badInitialCentroid_resetToPeak,base_PsfFlux_flag_badCentroid_resetToPeak,base_SdssShape_flag_badCentroid_resetToPeak,base_Variance_flag_badCentroid_resetToPeak,slot_Centroid_flag_resetToPeak,base_Blendedness_old,base_Blendedness_raw_flux,base_Blendedness_raw_flux_child,base_Blendedness_raw_flux_parent,base_Blendedness_abs_flux,base_Blendedness_abs_flux_child,base_Blendedness_abs_flux_parent,base_Blendedness_raw_child_xx,base_Blendedness_raw_child_yy,base_Blendedness_raw_child_xy,base_Blendedness_raw_parent_xx,base_Blendedness_raw_parent_yy,base_Blendedness_raw_parent_xy,base_Blendedness_abs_child_xx,base_Blendedness_abs_child_yy,base_Blendedness_abs_child_xy,base_Blendedness_abs_parent_xx,base_Blendedness_abs_parent_yy,base_Blendedness_abs_parent_xy,base_Blendedness_flag,base_Blendedness_flag_noCentroid,base_Blendedness_flag_noShape,base_SdssShape_xx,slot_Shape_xx,base_SdssShape_yy,slot_Shape_yy,base_SdssShape_xy,slot_Shape_xy,base_SdssShape_xxSigma,slot_Shap